In [452]:

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import mean_squared_error
import torch
from torch import nn
from torch.utils.data import TensorDataset, DataLoader
import torch.optim as optim


In [453]:
# 데이터 로드
data = pd.read_csv('C:\\AI\\diabetes.csv')

In [454]:
# 범주형 인코딩
# data['Sex'] = LabelEncoder().fit_transform(data['Sex']) 문자일 때만 사용

In [455]:
# Outcome (라벨) 제거 ( 새로운 라벨은 BMI )
data.drop(columns=['Outcome'], inplace=True)

# 입력과 타겟 분리
X = data.drop('BMI', axis=1).values
y = data['BMI'].values.astype(np.float32)

In [456]:
# 정규화
scaler = StandardScaler()
X = scaler.fit_transform(X)

# 데이터 분할
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [457]:

# TensorDataset으로 래핑
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32).view(-1, 1)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32).view(-1, 1)

train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32)


In [458]:

# 회귀 모델 정의
class RegressionModel(nn.Module):
    def __init__(self):
        super(RegressionModel, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(7, 64),  # feature 개수 일치하는지 확인
            nn.ReLU(),
            nn.Linear(64, 32),
            nn.ReLU(),
            nn.Linear(32, 1) # 출력층 개수 확인. 회귀면 1, 이진분류여도 1
        )

    def forward(self, x):
        return self.model(x)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = RegressionModel().to(device)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)


In [459]:

# 학습 루프
model.train()
for epoch in range(150):
    total_loss = 0
    for X_batch, y_batch in train_loader:
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)
        optimizer.zero_grad()
        output = model(X_batch)
        loss = criterion(output, y_batch)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    print(f"Epoch {epoch+1}, Loss: {total_loss / len(train_loader):.4f}")


Epoch 1, Loss: 1071.5732
Epoch 2, Loss: 1041.2189
Epoch 3, Loss: 950.2476
Epoch 4, Loss: 823.8314
Epoch 5, Loss: 621.8874
Epoch 6, Loss: 396.9876
Epoch 7, Loss: 212.8073
Epoch 8, Loss: 122.8702
Epoch 9, Loss: 102.7557
Epoch 10, Loss: 94.4049
Epoch 11, Loss: 85.8546
Epoch 12, Loss: 81.8448
Epoch 13, Loss: 78.9671
Epoch 14, Loss: 83.0225
Epoch 15, Loss: 72.2909
Epoch 16, Loss: 70.5516
Epoch 17, Loss: 67.9197
Epoch 18, Loss: 67.4644
Epoch 19, Loss: 64.6941
Epoch 20, Loss: 63.1622
Epoch 21, Loss: 63.0255
Epoch 22, Loss: 60.3520
Epoch 23, Loss: 60.0936
Epoch 24, Loss: 59.1754
Epoch 25, Loss: 55.5820
Epoch 26, Loss: 55.5953
Epoch 27, Loss: 57.2969
Epoch 28, Loss: 52.6284
Epoch 29, Loss: 52.5964
Epoch 30, Loss: 50.5754
Epoch 31, Loss: 49.5126
Epoch 32, Loss: 49.7887
Epoch 33, Loss: 48.3627
Epoch 34, Loss: 48.4094
Epoch 35, Loss: 48.5151
Epoch 36, Loss: 45.5654
Epoch 37, Loss: 45.3120
Epoch 38, Loss: 44.0320
Epoch 39, Loss: 46.4024
Epoch 40, Loss: 44.2381
Epoch 41, Loss: 45.4691
Epoch 42, Loss

In [460]:

# 평가
model.eval()
preds, actuals = [], []
with torch.no_grad():
    for X_batch, y_batch in test_loader:
        X_batch = X_batch.to(device)
        outputs = model(X_batch).cpu().numpy()
        preds.extend(outputs)
        actuals.extend(y_batch.numpy())

mse = mean_squared_error(actuals, preds)
print(f"Test MSE: {mse:.4f}")


Test MSE: 55.9844
